# Neighborly Functions App - Cloud Developer With MS Azure - Udacity Project II

## 1. Add configuration

In [ ]:
import configparser

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('azure.cfg'))

RESOURCE_GROUP                          = config.get('RESOURCE','RESOURCE_GROUP')
LOCATION                                = config.get('RESOURCE','LOCATION')
COSMOSDB_ACCOUNT_NAME                   = config.get("DATABASE","COSMOSDB_ACCOUNT_NAME")
COSMOSDB_KIND                           = config.get("DATABASE","COSMOSDB_KIND")
COSMOSDB_SERVER_VERSION                 = config.get("DATABASE","COSMOSDB_SERVER_VERSION")
COSMOSDB_DEFAULT_CONSISTENCY_LEVEL      = config.get("DATABASE","COSMOSDB_DEFAULT_CONSISTENCY_LEVEL")
COSMOSDB_ENABLE_AUTOMATIC_FAILOVER      = config.get("DATABASE","COSMOSDB_ENABLE_AUTOMATIC_FAILOVER")
COSMOSDB_DATBASE_NAME                   = config.get("DATABASE","COSMOSDB_DATABASE_NAME")
STORAGE_ACCOUNT_NAME                    = config.get("STORAGE","STORAGE_ACCOUNT_NAME")
STORAGE_SKU                             = config.get("STORAGE","STORAGE_SKU")
APP_NAME                                = config.get("APP","APP_NAME")
APP_SERVICE_PLAN_NAME                   = config.get("APP","APP_SERVICE_PLAN_NAME")
APP_SKU                                 = config.get("APP","APP_SKU")
APP_RUNTIME                             = config.get("APP","APP_RUNTIME")
FUNCTION_NAME                           = config.get("FUNCTION","FUNCTION_NAME")
FUNCTION_VERSION                        = config.get("FUNCTION","FUNCTION_VERSION")
FUNCTION_OS_TYPE                        = config.get("FUNCTION","FUNCTION_OS_TYPE")
FUNCTION_RUNTIME                        = config.get("FUNCTION","FUNCTION_RUNTIME")

In [ ]:
import os
os.environ["RESOURCE_GROUP"]                        = RESOURCE_GROUP
os.environ["LOCATION"]                              = LOCATION
os.environ["COSMOSDB_ACCOUNT_NAME"]                 = COSMOSDB_ACCOUNT_NAME
os.environ["COSMOSDB_KIND"]                         = COSMOSDB_KIND
os.environ["COSMOSDB_SERVER_VERSION"]               = COSMOSDB_SERVER_VERSION
os.environ["COSMOSDB_DEFAULT_CONSISTENCY_LEVEL"]    = COSMOSDB_DEFAULT_CONSISTENCY_LEVEL
os.environ["COSMOSDB_ENABLE_AUTOMATIC_FAILOVER"]    = COSMOSDB_ENABLE_AUTOMATIC_FAILOVER
os.environ["COSMOSDB_DATBASE_NAME"]                 = COSMOSDB_DATBASE_NAME
os.environ["STORAGE_ACCOUNT_NAME"]                  = STORAGE_ACCOUNT_NAME
os.environ["STORAGE_SKU"]                           = STORAGE_SKU
os.environ["APP_NAME"]                              = APP_NAME
os.environ["APP_SERVICE_PLAN_NAME"]                 = APP_SERVICE_PLAN_NAME
os.environ["APP_SKU"]                               = APP_SKU
os.environ["APP_RUNTIME"]                           = APP_RUNTIME
os.environ["FUNCTION_NAME"]                         = FUNCTION_NAME
os.environ["FUNCTION_VERSION"]                      = FUNCTION_VERSION
os.environ["FUNCTION_OS_TYPE"]                      = FUNCTION_OS_TYPE
os.environ["FUNCTION_RUNTIME"]                      = FUNCTION_RUNTIME

## 2. Create Resources

### 2.1. Login Azure CLI

In [ ]:
! az login

### 2.2. Create Resource Group

In [ ]:
! az group create \
    --name {RESOURCE_GROUP} \
    --location {LOCATION}

### 2.3. Create Storage Account

In [ ]:
! az storage account create \
    --name {STORAGE_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --location {LOCATION} \
    --sku {STORAGE_SKU}

### 2.4. Create Function App

In [ ]:
! az functionapp create \
    --name {FUNCTION_NAME}  \
    --storage-account {STORAGE_ACCOUNT_NAME} \
    --consumption-plan-location {LOCATION} \
    --resource-group {RESOURCE_GROUP} \
    --functions-version {FUNCTION_VERSION} \
    --os-type {FUNCTION_OS_TYPE} \
    --runtime {FUNCTION_RUNTIME}

### 2.5. Create a CosmosDB manage MongoDB instance

#### 2.5.1. Create CosmosDB

In [ ]:
! az cosmosdb create \
    -n {COSMOSDB_ACCOUNT_NAME} \
    -g {RESOURCE_GROUP} \
    --kind {COSMOSDB_KIND} \
    --server-version {COSMOSDB_SERVER_VERSION} \
    --default-consistency-level {COSMOSDB_DEFAULT_CONSISTENCY_LEVEL} \
    --enable-automatic-failover {COSMOSDB_ENABLE_AUTOMATIC_FAILOVER}

#### 2.5.2. Get CosmosDB Connection String

In [ ]:
! az cosmosdb list-connection-strings --name {COSMOSDB_ACCOUNT_NAME} --resource-group {RESOURCE_GROUP}

In [ ]:
connectionStrings = ! az cosmosdb keys list \
    --type connection-strings \
    --name {COSMOSDB_ACCOUNT_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --query 'connectionStrings[0].connectionString' \
    --output tsv
CONNECTION_STRING = str(connectionStrings[0])

#### 2.5.3. Create MongoDB Collections

1. Create advertisements collection

In [ ]:
! printf '[{"key": {"keys": ["_id"]}}]' > idxpolicy-$uniqueId.json

In [ ]:
! az cosmosdb mongodb collection create \
    -a neighborly-cosmosdb \
    -g azure-dev-udacity-prj2 \
    -d neighborlydb \
    -n advertisements \
    --shard '_id' \
    --throughput 400 \
    --idx @idxpolicy-$uniqueId.json

2. Create posts collection

In [ ]:
! az cosmosdb mongodb collection create \
    -a neighborly-cosmosdb \
    -g azure-dev-udacity-prj2 \
    -d neighborlydb \
    -n posts \
    --shard '_id' \
    --throughput 400 \
    --idx @idxpolicy-$uniqueId.json

3. If you want to delete a collection, use:

In [ ]:
! az cosmosdb mongodb collection delete \
    -a {COSMOSDB_ACCOUNT_NAME} \
    -g {RESOURCE_GROUP} \
    -d {COSMOSDB_DATABASE_NAME} \
    -n advertisements

4. View the collection

In [ ]:
! az cosmosdb mongodb collection list \
    --account-name neighborly-cosmosdb \
	--resource-group azure-dev-udacity-prj2 \
	--database-name neighborlydb

#### 2.5.4. Add data to collection

1. Add data to Advertisements Collection - Connection String from 2.5.2

In [ ]:
print(CONNECTION_STRING)

In [ ]:
! mongoimport \
    --uri=f"{CONNECTION_STRING}" \
    --db={COSMOSDB_DATABASE_NAME} \
    --collection=advertisements \
    --file='./sample_data/sampleAds.json' \
    --jsonArray

2. Add data to Posts Collection - Connection String from 2.5.2

In [ ]:
! mongoimport \
    --uri f"{CONNECTION_STRING}" \
    --collection posts \
    --file='./sample_data/samplePosts.json' \
    --jsonArray

In [ ]:
from pymongo import MongoClient

In [ ]:
# Create a MongoDB client
client = MongoClient(f"{CONNECTION_STRING}")

# Access the database and collection
db = client[f"neighborlydb"]

In [ ]:
advertisements = db["advertisements"]

result = advertisements.find({})

# Print out the query result
for doc in result:
    print(doc)


In [ ]:
posts = db["posts"]

result = posts.find({})

# Print out the query result
for doc in result:
    print(doc)


### 2.6. Create App Service for Front-End

#### 2.6.1. List Runtime Environment

In [ ]:
! az webapp list-runtimes

#### 2.6.2. Create App Service Plan

In [ ]:
! az appservice plan create \
    --name {APP_SERVICE_PLAN_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --location {LOCATION} \
    --sku {APP_SKU} \
    --is-linux

#### 2.6.3. Create App Service

In [ ]:
! az webapp create \
    --name {APP_NAME} \
    --resource-group {RESOURCE_GROUP} \
    --plan {APP_SERVICE_PLAN_NAME} \
    --runtime {APP_RUNTIME}

## 3. Connect Function with CosmosDB

### 3.1. Verify the resources

In [ ]:
! az functionapp list  --resource-group {RESOURCE_GROUP}

### 3.2. Connect

In [ ]:
! az functionapp config appsettings set \
  --name neighborly-function \
  --resource-group azure-dev-udacity-prj2 \
  --setting MyDbConnection="mongodb://neighborly-cosmosdb:E2Uoovo8hP4chc62kH25aelu9oJkYraYdsvfav1V3lO74SiOg7mZeqATqwBkmfSkDkHN4NpcKKzfACDb6SfMCA==@neighborly-cosmosdb.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@neighborly-cosmosdb@"

## 3. Deploy Functions

### 3.1. Test Functions at Local

In [ ]:
! func start

In [ ]:
! func azure functionapp publish {FUNCTION_NAME}

## 5. Delete Resources

### 5.1. Delete group - delete all resources belong to this group

In [ ]:
! az group delete -n {RESOURCE_GROUP} --verbose

### 5.2. Delete each resources

1. Delete CosmosDB

2. Delete Function App